# QuantumFold-Advantage Benchmarking Suite
This notebook demonstrates the benchmarking of QuantumFold-Advantage against AlphaFold3 and Boltz2, including real structural metrics and 3D confidence-colored visualizations.

## 1. Environment Setup

In [ ]:
# Clone the repository and install dependencies
!git clone https://github.com/your-repo/QuantumFold-Benchmark.git
%cd QuantumFold-Benchmark
!pip install -r requirements.txt
!pip install py3Dmol tabulate

import sys
import os
sys.path.append(os.path.abspath("src"))

## 2. Run Rigorous Benchmark
We execute the full suite on CASP15, Miniproteins, and IDRs.

In [ ]:
!python scripts/run_benchmarks.py

## 3. Results Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown

df = pd.read_csv('results/benchmark_results.csv')

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='dataset', y='rmsd', hue='model')
plt.title('RMSD Comparison across Datasets (Lower is Better)')
plt.ylabel('RMSD (Å)')
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='dataset', y='plddt', hue='model')
plt.title('Mean Model Confidence (pLDDT)')
plt.ylabel('pLDDT')
plt.show()

## 4. 3D Structural Visualization (AlphaFold-style)
Visualize predicted structures colored by pLDDT confidence (Blue > 90, Cyan 70-90, Yellow 50-70, Orange < 50).

In [ ]:
import py3Dmol

def view_structure(pdb_path):
    with open(pdb_path, 'r') as f:
        data = f.read()
    
    view = py3Dmol.view(width=800, height=600)
    view.addModel(data, 'pdb')
    
    # Standard AF pLDDT coloring
    view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    view.zoomTo()
    return view.show()

# Visualize the QuantumFold-Advantage prediction for a CASP15 target
print("Visualizing QuantumFold-Advantage Prediction for 1A2P:")
view_structure('results/1A2P_QuantumFold-Advantage.pdb')